In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.5QkNRFoXbQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.L9gwxqhATK/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.wbktg2oVrI/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!pip install selenium
!apt-get install chromium chromium-driver
!pip install undetected_chromedriver
!pip install webdriver_manager
!pip install pyvirtualdisplay
!sudo apt-get install xvfb

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:4 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:5 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [972 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:13 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:1

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chromium.webdriver import ChromiumDriver
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import undetected_chromedriver
from pyvirtualdisplay import Display

from selenium.webdriver.common.action_chains import ActionChains
from datetime import timedelta, datetime
import time

In [ ]:
display = Display(visible=0, size=(1920, 1200))  
display.start()
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('enable-automation')
options.add_argument('--disable-infobars')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-browser-side-navigation')
options.add_argument("--remote-debugging-port=9222")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
driver = webdriver.Chrome( options=options)
ac = ActionChains(driver) 
dataname = (datetime.now()).date()

In [ ]:
# список ссылок на сайт в различных странах
urls = [['United States',
         'https://www.glassdoor.com/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel'],
        ['United Kingdom',
         "https://www.glassdoor.co.uk/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Italy',
         "https://www.glassdoor.it/Lavoro/data-scientist-or-data-analyst-lavori-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Ireland',
         "https://www.glassdoor.ie/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Austria',
         "https://www.glassdoor.at/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Mexico',
         "https://www.glassdoor.com.mx/Empleo/data-scientist-or-data-analyst-empleos-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['France',
         "https://www.glassdoor.fr/Emploi/data-scientist-or-data-analyst-emplois-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Argentina',
         "https://www.glassdoor.com.ar/Empleo/data-scientist-or-data-analyst-empleos-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Belgium',
         "https://nl.glassdoor.be/Vacature/data-scientist-or-data-analyst-vacatures-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['India',
         "https://www.glassdoor.co.in/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Brazil',
         "https://www.glassdoor.com.br/Vaga/data-scientist-or-data-analyst-vagas-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Netherlands',
         "https://www.glassdoor.nl/Vacature/data-scientist-or-data-analyst-vacatures-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Australia',
         "https://www.glassdoor.com.au/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Germany',
         "https://www.glassdoor.de/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['New Zealand',
         "https://www.glassdoor.co.nz/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Canada',
         "https://www.glassdoor.ca/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Spain',
         "https://www.glassdoor.es/Empleo/data-scientist-or-data-analyst-empleos-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"],
        ['Switzerland',
         "https://de.glassdoor.ch/Job/data-scientist-or-data-analyst-jobs-SRCH_KO0,30.htm?fromAge=1&seniorityType=entrylevel"]
   
       ]

In [ ]:
def scan(cont):
    # поскольку на сайте не указывается конкретное время публикации вакансии, проставляем везде дату текущего дня
    today = datetime.now()
    date = today.date()
    
    # закрываем высплывающие окна, которые могут появиться, кликнув на основную часть страницы 
    try:
        ac.move_by_offset(1, 1).click().perform()                                                                                                                                                                                                                                                                                                                                
    except:
        print("err click  нет вспл окна")

    # забираем весь html страницы    
    sleep(1)
    soup=BeautifulSoup(driver.page_source,"lxml")

    # название компании
    try:
        employerName = soup.find("div",class_="css-87uc0g").text[:-3]
    except:
        employerName = "unknown"


    # имя вакансии
    try:
        jobTitle = soup.find("div",class_="css-1vg6q84 e1tk4kwz4").text
    except:
        jobTitle = "unknown"

    # месторасположение
    try:
        location = soup.find("div",class_="css-56kyx5 e1tk4kwz5").text
    except:
        location = "unknown"

    # зарплата
    try:
        detailSalary = driver.find_element(By.XPATH,"/html/body/div[2]/div/div/div/div/div[2]/section/div/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[4]/span").text
    except:
        detailSalary = "unknown"
    

    # ссылка
    try: 
        link = "https://www.glassdoor.com" + soup.find("button",class_="gd-ui-button applyButton css-1wuyo1a evpplnh1").get("data-job-url")

    except:
        link = "unknown"

    country = cont

    # текст вакансии
    try:    
        text2 = soup.find("div",class_="jobDescriptionContent").text
    except:
        text2 = "unknown"
    source = "GLASSDOOR"

    # сразу возвращаем df   
    return ([date,jobTitle, country,  employerName, detailSalary,location,link,text2,source])


In [ ]:
%%time
data=[]

# цикл по ссылкам
for urrr in range(len(urls)):
    driver.get(urls[urrr][1])
    sleep(3)

    # избавляемся от вспдывающих окон
    try:
        ac.move_by_offset(1, 1).click().perform() 
        sleep(3)
    except:
        pass

    # забираем весь html страницы
    soup=BeautifulSoup(driver.page_source,"lxml")
    
    
    # находим количество страниц и если нет вакансий по запросу пропускаем
    try:
        last_page = soup.find("div",class_="paginationFooter")
        last_page = int(last_page.text.split()[-1]) 
    except:
        print("continue ....") 
        continue

    # цикл по страницам
    for j in range(0,last_page): 
        sleep(2)
        soup=BeautifulSoup(driver.page_source,"lxml")
        sss = soup.find_all("li",class_="react-job-listing")
    
        for i in range(1,len(sss)+1): 
            try:
                sleep(2)
                knop= driver.find_element(By.XPATH,f'/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li[{i}]/div[2]/a')
                sleep(1)
                knop.click()
            except:
                sleep(1)
            try:
                data.append(scan(urls[urrr][0])) # передаем страну
            except:
                #print("eror scan")
                sleep(1)
        try:                                         
            next_page = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div/div[2]/section/article/div[2]/div/div[1]/button[4]')
            sleep(2) 
            next_page.click() # переход на следующую страницу
        except:
            pass

continue ....
continue ....
continue ....
continue ....
CPU times: user 21 s, sys: 702 ms, total: 21.7 s
Wall time: 12min 7s


In [ ]:
df=pd.DataFrame(data,columns=['data','jobTitle','country',
                              'employerName','detailSalary',
                              'location','links','text','source'])

In [ ]:
if df.duplicated().sum() > 0: # удалим дубликаты
    df = df.drop_duplicates()

In [ ]:
 # unknown в ссылках для вакансий, аплаить на которые нужно с самого сайта glassdoor а не с сайта работодателя
df

,data,jobTitle,country,employerName,detailSalary,location,links,text,source
0,2023-04-07,Data Scientist I - Python,United States,CNH Industrial,unknown,United States,https://www.glassdoor.com/partner/jobListing.h...,Overview:\n\nCNH Industrial is a world-class e...,GLASSDOOR
1,2023-04-07,Entry level Business /Data Analyst,United States,TRES,Employer Provided Salary:$30.00 - $35.00 Per Hour,"Dallas, TX",unknown,We are looking for a hands-on Business /Data A...,GLASSDOOR
2,2023-04-07,Jr Business Analyst/Jr Data Analyst,United States,TRES,Employer Provided Salary:$28.00 - $35.00 Per Hour,"Baltimore, MD",unknown,Job descriptionOur clients are looking for Jr ...,GLASSDOOR
3,2023-04-07,Junior Data Scientist,United States,New Directions Staff,Employer Provided Salary:$88K - $115K,Remote,https://www.glassdoor.com/partner/jobListing.h...,Are you Junior Data Scientist who is looking t...,GLASSDOOR
4,2023-04-07,Data Analyst I,United States,State of New Mexico,Employer Provided Salary:$50K - $80K,"Santa Fe, NM",https://www.glassdoor.com/partner/jobListing.h...,"Salary\n\n$24.11 - $38.58 Hourly\n$50,155 - $8...",GLASSDOOR
...,...,...,...,...,...,...,...,...,...
67,2023-04-07,Junior Frontend Developer (m/w/d),Germany,Statista GmbH,unknown,Hamburg,unknown,Aufgaben\nDu bist verantwortlich für die gesam...,GLASSDOOR
68,2023-04-07,(Junior) Produktmanager (m/w/d) App/Web Kfz-Ve...,Germany,CHECK24 Vergleichsportal,unknown,München,https://www.glassdoor.com/partner/jobListing.h...,KFZ-Versicherung\nMünchen\njetzt\nIm Überblick...,GLASSDOOR
69,2023-04-07,Junior Security Analyst,Canada,Okanagan College,Employer Provided Salary:$54K - $62K,Kelowna,https://www.glassdoor.com/partner/jobListing.h...,Position Information\nPosition Number S01082\n...,GLASSDOOR
70,2023-04-07,Junior Data Analyst - Banca H/F,Spain,Winning Consult,unknown,Madrid,unknown,Empresa\n\nSomos una empresa de consultoría qu...,GLASSDOOR


In [ ]:
# #loading to csv 
# from google.colab import drive
# drive.mount("/content/drive")
# df.to_csv (f'/content/drive/MyDrive/Glassdoor_{dataname}.csv', index = None, header=True, sep='\t', encoding='utf-8')

Mounted at /content/drive
